In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Customers and products
customer_ids = [101, 102, 103, 104, 105]
customer_names = ["Ali", "Sara", "Ahmed", "Hina", "Bilal"]
products = ["Keyboard", "Mouse", "Monitor", "Laptop", "Printer"]
price_dict = {"Keyboard": 5000, "Mouse": 1500, "Monitor": 15000, "Laptop": 80000, "Printer": 12000}

# Generate 75 transactions
num_transactions = 75
data = {
    "TransactionID": list(range(1, num_transactions + 1)),
    "CustomerID": [random.choice(customer_ids) for _ in range(num_transactions)],
    "CustomerName": [random.choice(customer_names) for _ in range(num_transactions)],
    "Product": [random.choice(products) for _ in range(num_transactions)],
    "Quantity": [random.randint(1, 5) for _ in range(num_transactions)],
    "Price": [],
    "Date": []
}

# Fill price and random date (last 30 days)
start_date = datetime.now() - timedelta(days=30)
for i in range(num_transactions):
    product = data["Product"][i]
    data["Price"].append(price_dict[product])
    random_date = start_date + timedelta(days=random.randint(0, 30))
    data["Date"].append(random_date.strftime("%Y-%m-%d"))

# Create DataFrame
df = pd.DataFrame(data)

# Save as CSV
df.to_csv("transactions.csv", index=False)
print("CSV file 'transactions.csv' created successfully!")


CSV file 'transactions.csv' created successfully!


In [2]:
import pandas as pd
import sqlite3

In [3]:
df = pd.read_csv("transactions.csv")

In [4]:
df.drop_duplicates(inplace=True)              
df.dropna(inplace=True)                         
df['Date'] = pd.to_datetime(df['Date'])  

In [5]:
df['TotalPrice'] = df['Quantity'] * df['Price']

In [6]:
conn = sqlite3.connect("data_warehouse.db")
cursor = conn.cursor()

In [7]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions (
    TransactionID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    CustomerName TEXT,
    Product TEXT,
    Quantity INTEGER,
    Price REAL,
    Date TEXT,
    TotalPrice REAL
)
''')

In [8]:
df.to_sql('transactions', conn, if_exists='replace', index=False)

75

In [16]:
query1 = "SELECT CustomerName, SUM(TotalPrice) AS TotalSpent FROM transactions GROUP BY CustomerID ORDER BY TotalSpent DESC"
result1 = pd.read_sql_query(query1, conn)

In [17]:
print("Total Spending per Customer:")
print(result1)

Total Spending per Customer:
  CustomerName  TotalSpent
0          Ali     1282000
1        Ahmed      847000
2        Ahmed      705000
3          Ali      661500
4          Ali      599500


In [18]:
query2 = "SELECT Product, SUM(TotalPrice) AS TotalSales FROM transactions GROUP BY Product ORDER BY TotalSales DESC"
result2 = pd.read_sql_query(query2, conn)

In [19]:
print("\nTotal Sales per Product:")
print(result2)


Total Sales per Product:
    Product  TotalSales
0    Laptop     2400000
1   Monitor      930000
2   Printer      468000
3  Keyboard      225000
4     Mouse       72000


In [20]:
query3 = "SELECT Date, SUM(TotalPrice) AS DailySales FROM transactions GROUP BY Date ORDER BY Date"
result3 = pd.read_sql_query(query3, conn)

In [21]:
print("\nDaily Sales Summary:")
print(result3)


Daily Sales Summary:
                   Date  DailySales
0   2025-12-08 00:00:00       88500
1   2025-12-09 00:00:00       23000
2   2025-12-10 00:00:00       65000
3   2025-12-11 00:00:00       30000
4   2025-12-12 00:00:00       80000
5   2025-12-13 00:00:00       30000
6   2025-12-14 00:00:00       30000
7   2025-12-15 00:00:00       83000
8   2025-12-16 00:00:00      116500
9   2025-12-17 00:00:00      324500
10  2025-12-18 00:00:00       79500
11  2025-12-19 00:00:00      436000
12  2025-12-20 00:00:00       22500
13  2025-12-21 00:00:00       48000
14  2025-12-22 00:00:00      204000
15  2025-12-23 00:00:00       15000
16  2025-12-25 00:00:00      243000
17  2025-12-26 00:00:00       20000
18  2025-12-27 00:00:00        3000
19  2025-12-28 00:00:00      160000
20  2025-12-29 00:00:00       40000
21  2025-12-30 00:00:00       48000
22  2025-12-31 00:00:00      160000
23  2026-01-01 00:00:00      483500
24  2026-01-02 00:00:00      110000
25  2026-01-03 00:00:00       90000
26  20

In [22]:
conn.close()